In [2]:
import tensorflow as tf
import numpy as np
from formatData import loadData
from sklearn.model_selection import train_test_split
from gan import GenAdvNetwork

In [3]:
"""
Loading data
Note than we can only specify absolute location of the raw data
"""
molRep2D, energies = loadData(12, "/home/panthibivek/thesis/GAN_pkg/data/traj.xyz")
#randomize the data and split it into training and test set
# unison_shuffle(molRep2D, energies)
X_train, X_test, y_train, y_test = train_test_split(molRep2D,energies,
                                   random_state=104, 
                                   test_size=0.3, 
                                   shuffle=True)

print("Training data size:", X_train.shape)
print("Test data size:", X_test.shape)

input data size: (10000, 78)
output data size: (10000,)
Training data size: (7000, 78)
Test data size: (3000, 78)


In [4]:
latent_dim_ = 32
epochs_ = 3
batch_size_ = 16
y_train = np.reshape(y_train, (-1, 1))
X_train = np.array(X_train)
X_train = X_train.astype(float)
X_train = np.reshape(X_train, (-1, 78, 1))
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.shuffle(buffer_size=100).batch(batch_size_)

2023-03-02 18:21:39.621077: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 18:21:39.621217: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-02 18:21:39.624425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libMIOpen.so'; dlerror: libMIOpen.so: cannot open shared object file: No such file or directory
2023-03-02 18:21:39.665584: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to down

In [5]:
GAN_model = GenAdvNetwork(latent_dim=latent_dim_, batch_size=batch_size_)
GAN_model.compile(
    generator_opt=tf.keras.optimizers.Adam(learning_rate=0.0001),
    discriminator_opt=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss_func=tf.keras.losses.BinaryCrossentropy(),
)
GAN_model.fit(dataset, epochs=epochs_)

2023-03-02 18:21:39.731992: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.732827: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.739914: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.740408: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.744566: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.745112: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.745924: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.746441: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.746926: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                442       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 13)                0         
                                                                 
 reshape (Reshape)           (None, 13, 1)             0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 39, 512)          2560      
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 39, 512)           0         
                                                                 
 batch_normalization (BatchN  (None, 39, 512)          2048      
 ormalization)                                           

2023-03-02 18:21:39.886778: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.887467: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.888360: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.888919: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.910086: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.910680: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.911496: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.911977: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 26, 64)            320       
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 26, 64)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 13, 128)           32896     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 13, 128)           0         
                                                                 
 batch_normalization_2 (Batc  (None, 13, 128)          512       
 hNormalization)                                                 
                                                                 
 flatten (Flatten)           (None, 1664)              0         
                                                     

2023-03-02 18:21:39.934147: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.934738: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.935629: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.936171: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.945939: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.946449: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.947292: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.947735: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39.964420: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:39

  1/438 [..............................] - ETA: 1:31 - g_loss: 0.7053 - d_loss: 0.4450

2023-03-02 18:21:40.136481: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:40.136645: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 31948800 exceeds 10% of free system memory.
2023-03-02 18:21:40.140876: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:40.141087: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 31948800 exceeds 10% of free system memory.
2023-03-02 18:21:40.171682: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 29286400 exceeds 10% of free system memory.
2023-03-02 18:21:40.184329: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:40.185161: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:40.186141: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2023-03-02 18:21:40.186854: I tensorfl

438/438 [==============================] - 59s 135ms/step - g_loss: 6.2094 - d_loss: 0.0630
Epoch 2/3
438/438 [==============================] - 59s 135ms/step - g_loss: 10.4284 - d_loss: 1.6873e-05
Epoch 3/3
438/438 [==============================] - 60s 136ms/step - g_loss: 11.3072 - d_loss: 7.1084e-06
